In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

SAMPLES_PER_GESTURE = 200

Model = None


In [19]:
def CreateModel():
    # create a NN with 2 layers of 16 neurons
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(6, SAMPLES_PER_GESTURE, )))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])   
        
    model.summary()
    
    return model

In [26]:
def PrepareModel(model):
    SAMPLES = 1000
    np.random.seed(1337)
    
    x_values = np.random.uniform(low=0, high=2*math.pi, size=(6, SAMPLES_PER_GESTURE, SAMPLES))
    # shuffle and add noise
    np.random.shuffle(x_values)
    y_values = np.sin(x_values)
    y_values += 0.1 * np.random.randn(*y_values.shape)

    # split into train, validation, test
    TRAIN_SPLIT =  int(0.6 * SAMPLES)
    TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])
   
    model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_validate, y_validate))
    
    return

In [21]:
def ConvertModel(model):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open("model.tflite", "wb") as f:
        f.write(tflite_model)
          

In [22]:
# Function: Convert some hex value into an array for C programming
def Hex2H(model, h_model_name):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    c_str = ''
    model_len = len(tflite_model)

    # Create header guard
    c_str += '#ifndef ' + h_model_name.upper() + '_H\n'
    c_str += '#define ' + h_model_name.upper() + '_H\n'

    # Add array length at top of file
    c_str += '\nunsigned int ' + h_model_name + '_len = ' + str(model_len) + ';\n'

    # Declare C variable
    c_str += 'unsigned char ' + h_model_name + '[] = {'
    hex_array = []
    for i, val in enumerate(tflite_model) :
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < model_len:
          hex_str += ','
        if (i + 1) % 12 == 0:
          hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + h_model_name.upper() + '_H\n'
        
    # Write TFLite model to a C source (or header) file
    with open(h_model_name + '.h', 'w') as file:
        file.write(c_str)   
    

In [23]:
Model = CreateModel()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6, 16)             3216      
                                                                 
 dense_4 (Dense)             (None, 6, 16)             272       
                                                                 
 dense_5 (Dense)             (None, 6, 1)              17        
                                                                 
Total params: 3,505
Trainable params: 3,505
Non-trainable params: 0
_________________________________________________________________


In [27]:
PrepareModel(Model)

Epoch 1/200


ValueError: in user code:

    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/root/miniconda3/envs/tinyml/lib/python3.7/site-packages/keras/engine/input_spec.py", line 264

In [ ]:
ConvertModel(Model)

In [ ]:
Hex2H(Model, "model")